In [ ]:
# ignore this section, written to be short as possible
function nb.JoinCsv { 
    [Alias('Csv', 'UL')]
    [cmdletbinding()]
    param(
        <# prefix string #>[Alias('op')][Parameter(Position=0)]$Prefix = '',
        <# suffix string #> [Alias('os')][Parameter(Position=1)]$Suffix = '',
        <# suffix string #> [Alias('Sep')][Parameter()]$Separator = ', ',
        <# suffix property #> [Alias('Prop', 'P')][Parameter()]$PropertyName = $Null,
        <# Actual Pipe #>[Parameter(ValueFromPipeline, Mandatory)][object[]]$InputObject,
        [switch]$Sort, [switch]$Unique, [int]$Pad = 0
        # [switch]$Qoute
    )
    begin {
        $items = [System.Collections.Generic.list[object]]::new()
    }
    process {
        foreach($cur in $InputObject) { $Items.add($cur)}
    } 
    end {
        $isUL = $PSCmdlet.MyInvocation.InvocationName -eq 'UL'

        $padding = "`n" * $Pad -join ''
        $Prefix = $Padding + $Prefix
        $Suffix = $Suffix + $Padding

        
        if($False -and 'original UL' -and $IsUL) {             
                $Prefix = $Prefix + "`n - "
                $Suffix = "`n - " + $Suffix
                $Separator = "`n - "
        }
        if($IsUL) {             
            if( (-not $Separator) -or $Separator -eq ', ') {
                $Separator = ' - '    
            }
            $Prefix = $Prefix + "`n${Separator}"
            $Suffix = "`n" + $Suffix
            # $Suffix = "`n${Separator}" + $Suffix # was
            $Separator = "`n${Separator}"
        }
        
        $splat = @{  'sep' = $Separator ; 'op' =  $Prefix ; 'os' = $Suffix }
        $IsRawText = $items[0] -is 'string'
        write-debug $IsRawText
        
        $filtered = if($Unique) { $Items | Sort -Unique }
        elseif($Sort) { $Items | sort }
        else { $items } 

        if($PropertyName -and (! $IsRawText)) { 
            $filtered | Join-String @splat -Prop $PropertyName
            # $Items | Join-String -sep ', ' -op:$Prefix -os $Suffix -Prop $PropertyName
        } else {
            $filtered | Join-String @splat
            # $Items | Join-String -sep ', ' -op:$Prefix -os $Suffix
        }
    }
}

('last executed: {0}' -f @((get-date).tostring('u')))
Import-Module NameIt -ea stop -passthru | Csv -Prop { $_.Name, $_.Version } -op 'modules = '
Import-Module NameIt -ea stop -passthru | Ul -Prop { $_.Name, $_.Version } -op 'modules = '
$PSVersionTable | join-string -op 'Pwsh: ' -Property PSVersion
Get-Command -m NameIT | Csv -prop Name -op 'NameIt Commands = '
Get-Command -m NameIT | ul -prop Name -op 'NameIt Commands = '



last executed: 2022-07-04 11:05:48Z
modules = NameIt 2.3.2
modules = 
 - NameIt 2.3.2

Pwsh: 7.2.4
NameIt Commands = Invoke-Generate, New-NameItTemplate
NameIt Commands = 
 - Invoke-Generate
 - New-NameItTemplate



<a name='nb-top' href='#'>Top</a>
### Using [github.com/dfinke/NameIt](https://github.com/dfinke/NameIt)
- part2: [Template type Summary Table](#nb-part2)
- part3: [New-NameItTemplate](#nb-part3)
<!-- - <a href='#nb-item-1'>Empty File</a>
- [Load Column, Row, hashtable](https://github.com/dfinke/ImportExcel/blob/master/FAQ/How%20to%20Read%20an%20Existing%20Excel%20File.md#other-common-operations) -->

In [ ]:
$Templates = @(
    '[person], [color]'
    '[address]'
    '[synonym cafe]_[syllable][syllable]'
    '####'
    '??-###-####-?##'
)
$Templates | %{ 
    foreach($i in 0..3) { 
    @(
        $_.PadLeft(20)
        ig $_
    ) -join ' = '
    }
}


   [person], [color] = Siena Dillon, green
   [person], [color] = Robert Allen, crimson
   [person], [color] = Maverick Terrell, crimson
   [person], [color] = Braiden Bright, yellow
           [address] = 10878 Bitrodsavaqji Div
           [address] = 7512 Ofumlip Streets
           [address] = 553 Deumsuklat Mntain
           [address] = 297 Arwek Ext
[synonym cafe]_[syllable][syllable] = coffeeshop_zooj
[synonym cafe]_[syllable][syllable] = coffeehouse_qooq
[synonym cafe]_[syllable][syllable] = eatinghouse_vaec
[synonym cafe]_[syllable][syllable] = coffeeshop_leex
                #### = 0570
                #### = 2994
                #### = 9545
                #### = 1271
     ??-###-####-?## = yc-919-0069-o93
     ??-###-####-?## = jb-349-8177-d83
     ??-###-####-?## = uy-972-8164-l42
     ??-###-####-?## = vg-187-1397-o94


In [ ]:

$Templates | %{ 
    $_ | Csv 'Template = "' -os '"'
    @(foreach($i in 0..3) { 
        Invoke-Generate $_     
    }) | UL -sep ' '
}

Template = "[person], [color]"

 Jaylynn Morris, yellow
 Walter Gentry, purple
 Cailyn Davidson, white
 Paityn Grant, amber
 
Template = "[address]"

 245 Tugotupugit Freeway
 57800 Boix Overpass
 141 Beraquje Knl
 812 Meokduy Plaza
 
Template = "[synonym cafe]_[syllable][syllable]"

 coffeehouse_seqin
 eatingplace_fifbu
 coffeeshop_nidoc
 coffeeshop_naoh
 
Template = "####"

 1410
 7055
 5317
 4096
 
Template = "??-###-####-?##"

 gl-427-6805-m01
 ue-806-6528-t07
 gd-112-6016-b53
 zl-010-5670-f82
 


<a href='#nb-top'>Go Top</a>
## <a name="nb-part2">Template Types Reference</a> 
 

| Template                | Description                                                                                        |
| ----------------------- | -------------------------------------------------------------------------------------------------- |
| `[alpha]`               | selects a random character (constrained by the -Alphabet parameter).                               |
| `[numeric]`             | selects a random numeric (constrained by the -Numbers parameter).                                  |
| `[vowel]`               | selects a vowel from a, e, i, o or u.                                                              |
| `[phoneticVowel]`       | selects a vowel sound, for example ou.                                                             |
| `[consonant]`           | selects a consonant from the entire alphabet.                                                      |
| `[syllable]`            | generates (usually) a pronouncable single syllable.                                                |
| `[synonym word]`        | finds a synonym to match the provided word.                                                        |
| `[person]`              | generate random name of female or male based on provided culture like <FirstName><Space><LastName> |
| `[person female]`       | generate random name of female based on provided culture like <FirstName><Space><LastName>.        |
| `[person female first]` |
| `[person female last]`  |
| `[person male]`         | generate random name of male based on provided culture like <FirstName><Space><LastName>.          |
| `[person male first]`   | .                                                                                                  |
| `[person male last]`    | .                                                                                                  |
| `[person both first]`   | .                                                                                                  |
| `[person both last]`    | .                                                                                                  |
| `[address]`             | generate a random street address. Formatting is biased to US currently.                            |
| `[space]`               | inserts a literal space. Spaces are striped from the templates string by default.                  |
 

In [ ]:

#from the docs
Invoke-Generate -alpha 'abc' -count 3 | ul


Invoke-Generate -count 5 "[synonym cafe]_[syllable][syllable]" | csv -op 'syllable: ' -pad 1 -sep "`n- "


 - cbaabbca
 - acbcaccb
 - acccbaba


syllable: eatingplace_nelef
- coffeehouse_cibpof
- coffeebar_jixi
- coffeeshop_rogom
- eatingplace_ijuy



In [ ]:
$Kind = New-NameItTemplate -sb {
    [PSCustomObject]@{
        Company = ''      
        Name    = ''
        Age     = 0
        address = ''
        state   = ''
        zip     = ''
    }        

}
Invoke-Generate $Kind -Count 3 -AsPSObject | Format-Table



state      zip   Age   Company Name          address
-----      ---   ---   ------- ----          -------
New Mexico 85001 35145 hwghkq  Sarai Murray  593 Wiugus Bayoo
New Jersey 21401 45893 kwbbbu  Mckinley Ford 610 Fudi Unions
Indiana    19901 81334 nrgegn  Dorian Hansen 763 Obgufok Falls



### Steps to use custom objects as templates

1. Create a `ScriptBlock` that returns a `[PSCustomObject]`
1. Call `Invoke-Generate $YourSB -Count 5`
1. Choose output type:
   1. raw text, 1 string per `-count` 
   2. or `-AsCsv` and `-AsTabDelimited`
   3. or a `[pscustomobject]`

> If a property name has has the value **name, zip, address, or state** the appropriate NameIT template is applied, **otherwise the type is inferred as numeric or alpha**.

```ps1
New-NameItTemplate {[PSCustomObject]@{Company="";Name=""}}
```

In [ ]:
'-AsPSObject' | csv -pad 1
Invoke-Generate $Kind -Count 2 -AsPSObject | Ft

'-AsCsv' | csv -pad 1
Invoke-Generate $Kind -Count 2 -AsCsv

'-AsTabDelimited' | csv -pad 1
Invoke-Generate $Kind -Count 2 -AsTabDelimited



-AsPSObject


state    zip   Age   Company Name         address
-----    ---   ---   ------- ----         -------
Michigan 50301 18147 ffyprn  King Frank   51545 Emak Rest
Arkansas 02108 95864 rvxmkt  Titus Lester 762 Josid Harbors


-AsCsv

"state","zip","Age","Company","Name","address"
"Mississippi","98501","72553","uzdiry","Lizbeth Roman","784 Oqve Sq"
"state","zip","Age","Company","Name","address"
"Washington","97301","60262","vjhklm","Reece Rivas","73136 Alxogop Roads"

-AsTabDelimited

"state"	"zip"	"Age"	"Company"	"Name"	"address"
"Michigan"	"23218"	"95040"	"jyibpy"	"Trevin Parsons"	"661 Tevdabjuv Ways"
"state"	"zip"	"Age"	"Company"	"Name"	"address"
"Nevada"	"25301"	"25690"	"fwpotb"	"Colton Valentine"	"714 Efkektiyes Hill"


<a href='#nb-top'>Go Top</a>
<h4><a name='nb-bottom'>See More</a>

- other `.inpynb` here
- Works well with [dfinke/ImportExcel](https://github.com/dfinke/ImportExcel)
- repo: [dfinke/NameIt](https://github.com/dfinke/NameIt)
  

In [ ]:
# Testing output
'
object properties as a hash summary:
'
Invoke-Generate $Kind -Count 2
| Foreach-Object { 
    $_ -split "\r?\n" | csv } | Join-String -op '@{ ' -os '} '
'
Almost hashtable literals:
'
Invoke-Generate $Kind -Count 2
| Foreach-Object { 
    $_ -split "\r?\n"
    | Join-String -prop {$_ | Join-String -DoubleQuote -sep ' ;  ' } } | Join-String -op '@{ ' -os '} '



object properties as a hash summary:

@{ Company=cmtrvv, Name=Kara Chaney, Age=76509, address=3283 Pasanuhciyho Ranch, state=North Dakota, zip=84101Company=xklsjy, Name=Ali Mayo, Age=79241, address=940 Hukxolomkulwaw Falls, state=Hawaii, zip=25301} 

Almost hashtable literals:

@{ "Company=hvxidl""Name=Shayna Hartman""Age=5403""address=64155 Vacehfizofic Clb""state=Massachusetts""zip=70801""Company=ithcsj""Name=Sawyer Villarreal""Age=6121""address=883 Xedfi Court""state=West Virginia""zip=43201"} 


In [ ]:
New-NameItTemplate {[PSCustomObject]@{Company="";Name=""}}

"round trip should not fail"
[PSCustomObject]@{ Company=$null; Name="" } 
| ConvertTo-Json | convertfrom-json
| ConvertTo-Json



Company=[alpha 6]
Name=[person]
round trip should not fail
{
  "Company": null,
  "Name": ""
}


In [ ]:
# Can json directly easily work?
$json = @'
{
    "Name": "",
    "State": "",
    "EmployeId": 0
}
'@
$Template = @{}

$Obj = $Json | ConvertFrom-Json

$Template.FromJson = New-NameItTemplate -sb { $obj }
$Template.FromCommand = New-NameItTemplate {
    [PSCustomObject]@{
        Name=""
        State=""
        EmployeeId=""
    }
}

'from a [PSCO]' | csv -pad 1
$Template.FromCommand

'from Json' | Csv -Pad 1
$Template.FromJson


from a [PSCO]

Name=[person]
State=[state]
EmployeeId=[alpha 6]

from Json

Name=[person]
State=[state]
EmployeId=[numeric 5]


<a href='#nb-top'>Go Top</a>
## <a name="nb-part3">New-NameItTemplate</a> 
 
